In [1]:
%pip install -q -r ../../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
print(load_dotenv('../../.env'))
print(os.environ['LANGSMITH_PROJECT'])
os.environ['LANGSMITH_TRACING']="true"
os.environ['USER_AGENT'] = 'myagent'
QDRANT_URL="http://localhost:6333"

True
agentic-ops


In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

retriever = QdrantVectorStore(
            client=QdrantClient(url=QDRANT_URL),
            collection_name="documentations",
            embedding=OpenAIEmbeddings(model="text-embedding-3-small")
        ).as_retriever()

In [4]:
retriever.invoke("what is huggingface ? ")[0]

Document(metadata={'filename': 'model_sharing.md', '_id': '15d6194e-4457-40a1-9473-b5a4e86ece82', '_collection_name': 'documentations'}, page_content='library installed. This library allows you to programmatically interact with the Hub.\\n\\n```bash\\n\\npip install huggingface_hub\\n\\nThen use `notebook_login` to sign-in to the Hub, and follow the link [here](https://huggingface.co/settings/token) to generate a token to login with:\\n\\n>>> from huggingface_hub import notebook_login\\n\\n>>> notebook_login()\\n\\n## Convert a model for all frameworks\\n\\nTo ensure your model can be used by someone working with a different framework, we recommend you convert and upload your model with both PyTorch and TensorFlow checkpoints. While users are still able to load your model from a different framework if you skip this step, it will be slower because \\ud83e\\udd17 Transformers will need to convert the checkpoint on-the-fly.\\n\\nConverting a checkpoint for another framework is easy. Make 

### Simple RAG application

In [8]:
from langsmith import traceable
from openai import OpenAI
from typing import List

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return OpenAI().chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [9]:
question = "What is huggingface used for?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"sample_runtime": "metadata added"}})
print(ai_answer)

Hugging Face is used for sharing and accessing machine learning models, particularly in natural language processing and other AI tasks. It provides a platform (Model Hub) where users can upload, download, and collaborate on models, facilitating easier model deployment and experimentation. Additionally, it offers libraries like Transformers for building and fine-tuning models across various frameworks.


https://smith.langchain.com/